In [1]:
import numpy as np
import sympy as sp
import matplotlib.pyplot as plt
import matplotlib.cm as cm

import sys
import os
import torch as pt
from tqdm import tqdm

# Add the project root
sys.path.append(os.path.abspath('../../'))

from src.useful_functions import read_dirs_paths
from src.isokann.modules3 import *

# For matplotlib
font = {'size'   : 10}
plt.rc('font', **font)
in2cm = 1/2.54  # centimeters in inches

# For reproducibility
np.random.seed(0)
pt.manual_seed(0)

# Read directory paths
read_dirs_paths('dir_paths.txt', globals())

device = pt.device("cuda" if pt.cuda.is_available() else "cpu")
print("")
print(device)


Created variables:
inp_dir = input/
out_trajectories1 = output/trajectories/
out_trajectories2 = output/trajectories/openmm_files/
out_trajectories3 = output/trajectories/openmm_files/initial_states/
out_trajectories4 = output/trajectories/openmm_files/final_states/
out_isokann = output/isokann/
out_mokito = output/mokito/

cuda


In [2]:
# Load initial and final states and convert to torch
D0 = pt.load(out_trajectories1 + 'BAT_0.pt', map_location=device)
DT = pt.load(out_trajectories1 + 'BAT_t.pt', map_location=device)

Npoints = D0.shape[0]
Ndims   = D0.shape[1]

Nfinpoints  = DT.shape[1]
Nframes     = DT.shape[3]


frame = 9
Dt = pt.clone(DT[frame,:,:,:])

print(Dt.shape)

torch.Size([6000, 10, 1716])


In [3]:
NN_epochs = [5, 10 ,15]


NN_nodes =          [(Ndims, int(2*Ndims/3), 1), 
                      (Ndims, int(2*Ndims/3), int(Ndims/3), 1),
                      (Ndims, int(Ndims/2), 1), 
                      (Ndims, int(Ndims/2),int(Ndims/4), 1)]

NN_lr = [ 0.01,
          0.005,
          0.001]

NN_wd  = [ 0.001,
           0.005,
           0.0001]

NN_bs  = [50, 100, 200]

NN_patience = [2, 3, 5]

NN_act_fun = ['leakyrelu']

best_hyperparams, best_val_loss = random_search(D0,
                                                Dt,
                                                NN_epochs,
                                                NN_nodes,
                                                NN_lr,
                                                NN_wd,
                                                NN_bs,
                                                NN_patience,
                                                NN_act_fun, 
                                                search_iterations=30, #15
                                                test_size = 0.2)


print("The best hyperparameters are:", best_hyperparams)
print("The best validation loss is:",  best_val_loss)



  0%|          | 0/30 [00:00<?, ?it/s]

 
Nepochs = 10
Nodes = [1716  858  429    1]
Learning rate = 0.01
Weight decay = 0.005
Batch size = 200
Patience = 3
Activation function = leakyrelu
Validation loss: 

  3%|▎         | 1/30 [00:24<11:55, 24.66s/it]

tensor(0.0006, device='cuda:0')
Convergence: 1.000576081598026
 
Nepochs = 10
Nodes = [1716  858  429    1]
Learning rate = 0.005
Weight decay = 0.0001
Batch size = 50
Patience = 5
Activation function = leakyrelu


  7%|▋         | 2/30 [00:49<11:31, 24.70s/it]

...

Validation loss: tensor(0.0023, device='cuda:0')
Convergence: 1.0008726987183822
 
Nepochs = 5
Nodes = [1716  858    1]
Learning rate = 0.001
Weight decay = 0.005
Batch size = 50
Patience = 5
Activation function = leakyrelu


 97%|█████████▋| 29/30 [07:36<00:18, 18.84s/it]

Validation loss: tensor(0.0063, device='cuda:0')
Convergence: 0.99983538887892
 
Nepochs = 15
Nodes = [1716 1144  572    1]
Learning rate = 0.01
Weight decay = 0.005
Batch size = 200
Patience = 3
Activation function = leakyrelu


100%|██████████| 30/30 [07:45<00:00, 15.50s/it]

Validation loss: tensor(0.0012, device='cuda:0')
Convergence: 1.0007163770039467
The best hyperparameters are: {'Nepochs': 15, 'nodes': array([1716,  858,  429,    1]), 'learning_rate': 0.01, 'weight_decay': 0.005, 'batch_size': 100, 'patience': 5, 'act_fun': 'leakyrelu'}
The best validation loss is: tensor(0.0005, device='cuda:0')


In [4]:
import pickle
# Save to a file
with open(out_isokann + 'hyperparameters.pkl', 'wb') as file:
    pickle.dump(best_hyperparams, file)